In [1]:
import tensorflow as tf
import numpy as np
import os

tf.__version__

'1.2.1'

In [25]:
# Config:

raw_img_dir = "./data/cifar-10-batches-bin/"
train_files = ["data_batch_1.bin", "data_batch_2.bin", "data_batch_3.bin", "data_batch_4.bin"]
valid_files = ["data_batch_5.bin"]
test_files = ["test_batch.bin"]

MODEL_DIR = "./models/"
MODEL_NAME = "conv1"


In [3]:
# Input functions:


IMAGE_SIZE = 32
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000 #50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000 #10000

def read_cifar10(filename_queue):

    class CIFAR10Record(object):
        pass
    result = CIFAR10Record()
    
    label_bytes = 1  # 2 for CIFAR-100
    result.height = IMAGE_SIZE
    result.width = IMAGE_SIZE
    result.depth = 3
    image_bytes = result.height * result.width * result.depth
    record_bytes = label_bytes + image_bytes

    reader = tf.FixedLengthRecordReader(record_bytes=record_bytes)
    result.key, value = reader.read(filename_queue)
    record_bytes = tf.decode_raw(value, tf.uint8)
    result.label = tf.cast(tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32)
    depth_major = tf.reshape(
      tf.strided_slice(record_bytes, [label_bytes],
                       [label_bytes + image_bytes]),
      [result.depth, result.height, result.width])
    result.uint8image = tf.transpose(depth_major, [1, 2, 0])

    return result


def _generate_image_and_label_batch(image, label, min_queue_examples, batch_size, shuffle):
    num_preprocess_threads = 16
    if shuffle:
        images, label_batch = tf.train.shuffle_batch(
            [image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + num_preprocess_threads * batch_size,
            min_after_dequeue=min_queue_examples)
    else:
        images, label_batch = tf.train.batch(
            [image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + num_preprocess_threads * batch_size)

    tf.summary.image('images', images)
    return images, tf.reshape(label_batch, [batch_size])

'''
Data Augmentation Pipeline
'''

def distorted_inputs(data_dir, batch_size):

    with tf.name_scope("Input"):

        filenames = [os.path.join(data_dir,fn) for fn in train_files]
        filename_queue = tf.train.string_input_producer(filenames, capacity = batch_size*16)
        read_input = read_cifar10(filename_queue)
        reshaped_image = tf.cast(read_input.uint8image, tf.float32)

        height = IMAGE_SIZE
        width = IMAGE_SIZE

        distorted_image = tf.random_crop(reshaped_image, [height, width, 3])
        distorted_image = tf.image.random_flip_left_right(distorted_image)
        distorted_image = tf.image.random_brightness(distorted_image,
                                                   max_delta=63)
        distorted_image = tf.image.random_contrast(distorted_image,
                                                 lower=0.2, upper=1.8)
        # image whitening
        float_image = tf.image.per_image_standardization(distorted_image) 

        float_image.set_shape([height, width, 3])
        read_input.label.set_shape([1])

        min_fraction_of_examples_in_queue = 0.4
        min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN * min_fraction_of_examples_in_queue)
        print ('Filling queue with %d CIFAR images before starting to train. '
             'This will take a few minutes.' % min_queue_examples)

        return _generate_image_and_label_batch(float_image, read_input.label, min_queue_examples, batch_size, shuffle=True)

def get_train():
    return distorted_inputs(raw_img_dir, batch_size=128)

def get_valid():
    return distorted_inputs(raw_img_dir, batch_size=128)

In [4]:
# Model function:

CONV1_SIZE = 5
CONV1_FEATS = 32
CONV2_SIZE = 5
CONV2_FEATS = 32
FC_SIZE = 200
OUT_CLASSES = 10

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x, name):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

def weight_var(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.get_variable(name=name, initializer=initial)

def bias_var(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def model_fn(features, targets, mode, params):
    
    with tf.name_scope("MODEL_FUNCTION"):
        
        ##############################
        # First convolutional layer: #
        ##############################

        with tf.name_scope("Layer_1"):
            
            W_conv1 = weight_var([CONV1_SIZE, CONV1_SIZE, 3, CONV1_FEATS], "w1")
            b_conv1 = bias_var([CONV1_FEATS])
            h_conv1 = tf.nn.relu(conv2d(features, W_conv1) + b_conv1)
            h_pool1 = max_pool_2x2(h_conv1, "hpool1")

        ###############################
        # Second convolutional layer: #
        ###############################

        with tf.name_scope("Layer_2"):
            W_conv2 = weight_var([CONV2_SIZE, CONV2_SIZE, CONV1_FEATS, CONV2_FEATS], "w2")
            b_conv2 = bias_var([CONV2_FEATS])
            h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
            h_pool2 = max_pool_2x2(h_conv2, "hpool2")

        ############################
        # Densely connected layer: #
        ############################

        with tf.name_scope("Dense_1"):

            new_img_size = IMAGE_SIZE / 4
            W_fc1 = weight_var([new_img_size * new_img_size * CONV2_FEATS, FC_SIZE], "wfc1")
            b_fc1 = bias_var([FC_SIZE])

            h_pool2_flat = tf.reshape(h_pool2, [-1, new_img_size * new_img_size * CONV2_FEATS])
            h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

        ##################
        # Dropout layer: #
        ##################

        with tf.name_scope("Dropout_layer"):

            keep_prob = params["dropout_keep_prob"]
            h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

        ##################
        # Readout layer: #
        ##################

        with tf.name_scope("Readout_layer"):

            W_fc2 = weight_var([FC_SIZE, OUT_CLASSES], "wfc2")
            b_fc2 = bias_var([OUT_CLASSES])

            y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

            y_true = tf.one_hot(targets, OUT_CLASSES)

        #########
        # Loss: #
        #########

        with tf.name_scope("Loss"):

            # Cross-entropy loss:
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_conv))

        #############
        # Accuracy: #
        #############

        pred_vals = tf.argmax(y_conv, 1)
        correct_vals = tf.argmax(y_true, 1)
        acc = tf.reduce_mean(tf.cast(tf.equal(pred_vals, correct_vals), tf.float64))


        with tf.name_scope("Train"):

            # Training subgraph:
            global_step = tf.train.get_global_step()
            optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])
            train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))


        ###############################
        # Summaries and eval metrics: #
        ###############################
        with tf.name_scope('summaries'):
            tf.summary.scalar("Loss", loss)
            tf.summary.scalar("Accuracy", acc)
        eval_metric_ops = {
            "Accuracy": acc
        }

        # Return a "ModelFnOps" instance:
        preds = {'y_conv': y_conv, 'h_pool1': h_pool1, 'h_pool2': h_pool2}
        return tf.contrib.learn.ModelFnOps(mode=mode,
                                           predictions=preds,
                                           loss=loss,
                                           train_op=train,
                                           eval_metric_ops=eval_metric_ops)


In [5]:
import tensorflow.contrib.learn as tflearn
from tensorflow.contrib.learn.python.learn import learn_runner
import tensorflow.contrib.metrics as metrics

In [23]:
MODEL_DIR

'./models/'

In [22]:
MODEL_NAME

'conv1'

In [8]:
# Clear model dir:
#!rm -r $MODEL_DIR

In [31]:
# Train the estimator:
tf.logging.set_verbosity(v=tf.logging.INFO)
model_params = {"learning_rate": 0.0003, "dropout_keep_prob": 0.8}

# Clear model dir:
#!rm -r $MODEL_DIR

model_dir = os.path.join(MODEL_DIR, MODEL_NAME)

my_estimator = tf.contrib.learn.Estimator(model_fn=model_fn, params=model_params, model_dir=model_dir)
#my_estimator.fit(input_fn=get_train, steps=5000)


def experiment_fn(output_dir):
    return tflearn.Experiment(
        my_estimator,
        train_input_fn=get_train,
        eval_input_fn=get_valid,
        train_steps=100,
        eval_steps=10,
        train_steps_per_iteration=500
    )

learn_runner.run(experiment_fn, output_dir="/tt/1", schedule="continuous_train_and_eval")

print "done!"

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x113cd67d0>, '_model_dir': './models/conv1', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_master': ''}
INFO:tensorflow:Training model for 500 steps
Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./models/conv1/model.ckpt-2001
INFO:tensorflow:Saving checkpoints for 2002 into ./models/conv1/model.ckpt.
INFO:tensorflow:loss = 1.0

In [27]:
my_estimator.__dict__

{'_config': <tensorflow.contrib.learn.python.learn.estimators.run_config.RunConfig at 0x113cd6850>,
 '_device_fn': None,
 '_feature_engineering_fn': <function tensorflow.contrib.learn.python.learn.estimators.estimator._identity_feature_engineering_fn>,
 '_features_info': TensorSignature(dtype=tf.float32, shape=TensorShape([Dimension(128), Dimension(32), Dimension(32), Dimension(3)]), is_sparse=False),
 '_graph': <tensorflow.python.framework.ops.Graph at 0x113cd6690>,
 '_labels_info': TensorSignature(dtype=tf.int32, shape=TensorShape([Dimension(128)]), is_sparse=False),
 '_model_dir': './models/conv1',
 '_model_fn': <function __main__.model_fn>,
 '_session_config': allow_soft_placement: true,
 'params': {'dropout_keep_prob': 0.7, 'learning_rate': 0.001}}

In [39]:
import matplotlib.pyplot as plt
%matplotlib inline

MODEL_PATH = "./models/conv1"
IMAGE_PATH = "./data/cifar-10-batches-bin/data_batch_1.bin"

def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict

def reshape_img(img_arr):
    # https://stackoverflow.com/questions/20341614/numpy-array-row-major-and-column-major
    return img_arr.reshape([1024, 3], order='F').reshape([32, 32, 3]) / 255.

def plot_img(img_arr):
    plt.imshow(img_arr)
    plt.show()
    
    
tmp = unpickle(IMAGE_PATH)
data, labels = tmp["data"], tmp["labels"]
    
img_index = 1 # Some random image
img = reshape_img(data[img_index])
    
plot_img(img)

UnpicklingError: invalid load key, ''.

In [ ]:
def single_img_fn():
    return tf.cast(tf.expand_dims(tf.constant(img), 0), tf.float32)

single_img_fn()

x = list(my_estimator.predict(input_fn=single_img_fn))

In [ ]:
prediction = x[0]['y_conv']
hpool1 = x[0]['h_pool1']
hpool2 = x[0]['h_pool2']

prediction

In [ ]:
print "Shape:", hpool1.shape
for i in range(3):
    grid = hpool1[:, :, i]
    plt.imshow(grid)
    plt.show()

In [ ]:
print "Shape:", hpool2.shape
#for i in range(hpool2.shape[2]):
for i in range(3):    
    grid = hpool2[:, :, i]
    plt.imshow(grid)
    plt.show()